In [1]:
import os

os.environ["KERAS_BACKEND"] = "tensorflow"

import pathlib
import random
import string
import re
import numpy as np

import tensorflow.data as tf_data
import tensorflow.strings as tf_strings

import keras
from keras import layers
import tensorflow.keras.backend as ops  # Import ops from tensorflow.keras.backend
from keras.layers import TextVectorization

In [3]:
text_file = "/content/English-Spanish.txt"

In [5]:
with open(text_file) as f:
    lines = f.read().split("\n")[:-1]

text_pairs = []
for line in lines:
    parts = line.split("\t")
    eng = parts[0]
    spa = parts[1]
    spa = "[start] " + spa + " [end]"
    text_pairs.append((eng, spa))


In [18]:
text_pairs[0]

('I hugged Tom.', '[start] Abracé a Tom. [end]')

In [10]:
for _ in range(5):
    print(random.choice(text_pairs))

("I don't care what he says.", '[start] No me importa lo que diga él. [end]')
("It's simple and intuitive.", '[start] Es sencillo e intuitivo. [end]')
('"Why are you going to Japan?" "To attend a conference in Tokyo."', '[start] "¿Para qué vas a Japón?" "Para asistir a una conferencia en Tokio." [end]')
('Something in his face really reminded me of an old boyfriend of mine.', '[start] Algo en su cara me recordaba verdaderamente a un viejo novio mío. [end]')
("Please don't cry again.", '[start] Por favor no llores de nuevo. [end]')


In [11]:
random.shuffle(text_pairs)
num_val_samples = int(0.15 * len(text_pairs))
num_train_samples = len(text_pairs) - 2 * num_val_samples
train_pairs = text_pairs[:num_train_samples]
val_pairs = text_pairs[num_train_samples : num_train_samples + num_val_samples]
test_pairs = text_pairs[num_train_samples + num_val_samples :]

print(f"{len(text_pairs)} total pairs")
print(f"{len(train_pairs)} training pairs")
print(f"{len(val_pairs)} validation pairs")
print(f"{len(test_pairs)} test pairs")

139705 total pairs
97795 training pairs
20955 validation pairs
20955 test pairs


In [61]:
strip_chars = string.punctuation + "¿"
strip_chars = strip_chars.replace("[", "")
strip_chars = strip_chars.replace("]", "")

vocab_size = 15000
sequence_length = 20
batch_size = 64


def custom_standardization(input_string):
    lowercase = tf_strings.lower(input_string)
    # Keep digits and /
    return tf_strings.regex_replace(lowercase, "[^a-z0-9/]", "")

eng_vectorization = TextVectorization(
    max_tokens=vocab_size,
    output_mode="int",
    output_sequence_length=sequence_length,
)
spa_vectorization = TextVectorization(
    max_tokens=vocab_size,
    output_mode="int",
    output_sequence_length=sequence_length + 1,
    # Remove the standardize argument
)
train_eng_texts = [pair[0] for pair in train_pairs]
train_spa_texts = [pair[1] for pair in train_pairs]
eng_vectorization.adapt(train_eng_texts)
spa_vectorization.adapt(train_spa_texts)

In [62]:
 def format_dataset(eng, spa):
    eng = eng_vectorization(eng)
    spa = spa_vectorization(spa)
    return (
        {
            "encoder_inputs": eng,
            "decoder_inputs": spa[:, :-1],
        },
        spa[:, 1:],
    )


def make_dataset(pairs):
    eng_texts, spa_texts = zip(*pairs)
    eng_texts = list(eng_texts)
    spa_texts = list(spa_texts)
    dataset = tf_data.Dataset.from_tensor_slices((eng_texts, spa_texts))
    dataset = dataset.batch(batch_size)
    dataset = dataset.map(format_dataset)
    return dataset.cache().shuffle(2048).prefetch(16)


train_ds = make_dataset(train_pairs)
val_ds = make_dataset(val_pairs)

In [63]:
 for inputs, targets in train_ds.take(1):
    print(f'inputs["encoder_inputs"].shape: {inputs["encoder_inputs"].shape}')
    print(f'inputs["decoder_inputs"].shape: {inputs["decoder_inputs"].shape}')
    print(f"targets.shape: {targets.shape}")

inputs["encoder_inputs"].shape: (64, 20)
inputs["decoder_inputs"].shape: (64, 20)
targets.shape: (64, 20)


In [64]:
 for batch in train_ds.take(1):  # Take one batch from the dataset
    inputs, targets = batch
    encoder_inputs = inputs['encoder_inputs']
    decoder_inputs = inputs['decoder_inputs']
    decoder_targets = targets

    print("Encoder inputs (vectorized):")
    print(encoder_inputs.numpy())

    print("Decoder inputs (vectorized):")
    print(decoder_inputs.numpy())

    print("Decoder targets (vectorized):")
    print(decoder_targets.numpy())

Encoder inputs (vectorized):
[[   2   18  537 ...    0    0    0]
 [   3  400   15 ...    0    0    0]
 [  13   53   97 ...    0    0    0]
 ...
 [  19  176  295 ...    0    0    0]
 [   9 1080  311 ...    0    0    0]
 [   2  117   25 ...    0    0    0]]
Decoder inputs (vectorized):
[[   2   69 1580 ...    0    0    0]
 [   2   10  219 ...    0    0    0]
 [   2    6   14 ...    0    0    0]
 ...
 [   2   23  247 ...    0    0    0]
 [   2 5298    4 ...    0    0    0]
 [   2  954    5 ...    0    0    0]]
Decoder targets (vectorized):
[[  69 1580   46 ...    0    0    0]
 [  10  219   50 ...    0    0    0]
 [   6   14 2706 ...    0    0    0]
 ...
 [  23  247 1127 ...    0    0    0]
 [5298    4  788 ...    0    0    0]
 [ 954    5   48 ...    0    0    0]]


In [65]:
import tensorflow as tf
from tensorflow.keras import layers

class TransformerEncoder(layers.Layer):
    def __init__(self, embed_dim, dense_dim, num_heads, **kwargs):
        super().__init__(**kwargs)
        self.embed_dim = embed_dim
        self.dense_dim = dense_dim
        self.num_heads = num_heads
        self.attention = layers.MultiHeadAttention(
            num_heads=num_heads, key_dim=embed_dim
        )
        self.dense_proj = tf.keras.Sequential(
            [
                layers.Dense(dense_dim, activation="relu"),
                layers.Dense(embed_dim),
            ]
        )
        self.layernorm_1 = layers.LayerNormalization()
        self.layernorm_2 = layers.LayerNormalization()
        self.supports_masking = True

    def call(self, inputs, mask=None):
        if mask is not None:
            padding_mask = tf.cast(mask[:, None, :], dtype="int32")
        else:
            padding_mask = None

        attention_output = self.attention(
            query=inputs, value=inputs, key=inputs, attention_mask=padding_mask
        )
        proj_input = self.layernorm_1(inputs + attention_output)
        proj_output = self.dense_proj(proj_input)
        return self.layernorm_2(proj_input + proj_output)

    def get_config(self):
        config = super().get_config()
        config.update(
            {
                "embed_dim": self.embed_dim,
                "dense_dim": self.dense_dim,
                "num_heads": self.num_heads,
            }
        )
        return config


class PositionalEmbedding(layers.Layer):
    def __init__(self, sequence_length, vocab_size, embed_dim, **kwargs):
        super().__init__(**kwargs)
        self.token_embeddings = layers.Embedding(
            input_dim=vocab_size, output_dim=embed_dim
        )
        self.position_embeddings = layers.Embedding(
            input_dim=sequence_length, output_dim=embed_dim
        )
        self.sequence_length = sequence_length
        self.vocab_size = vocab_size
        self.embed_dim = embed_dim

    def call(self, inputs):
        length = tf.shape(inputs)[-1]
        positions = tf.range(0, length, 1)
        embedded_tokens = self.token_embeddings(inputs)
        embedded_positions = self.position_embeddings(positions)
        return embedded_tokens + embedded_positions

    def compute_mask(self, inputs, mask=None):
        if mask is None:
            return None
        else:
            return tf.not_equal(inputs, 0)

    def get_config(self):
        config = super().get_config()
        config.update(
            {
                "sequence_length": self.sequence_length,
                "vocab_size": self.vocab_size,
                "embed_dim": self.embed_dim,
            }
        )
        return config


class TransformerDecoder(layers.Layer):
    def __init__(self, embed_dim, latent_dim, num_heads, **kwargs):
        super().__init__(**kwargs)
        self.embed_dim = embed_dim
        self.latent_dim = latent_dim
        self.num_heads = num_heads
        self.attention_1 = layers.MultiHeadAttention(
            num_heads=num_heads, key_dim=embed_dim
        )
        self.attention_2 = layers.MultiHeadAttention(
            num_heads=num_heads, key_dim=embed_dim
        )
        self.dense_proj = tf.keras.Sequential(
            [
                layers.Dense(latent_dim, activation="relu"),
                layers.Dense(embed_dim),
            ]
        )
        self.layernorm_1 = layers.LayerNormalization()
        self.layernorm_2 = layers.LayerNormalization()
        self.layernorm_3 = layers.LayerNormalization()
        self.supports_masking = True

    def call(self, inputs, encoder_outputs, mask=None):
        causal_mask = self.get_causal_attention_mask(inputs)
        if mask is not None:
            padding_mask = tf.cast(mask[:, None, :], dtype="int32")
            padding_mask = tf.minimum(padding_mask, causal_mask)
        else:
            padding_mask = None

        attention_output_1 = self.attention_1(
            query=inputs, value=inputs, key=inputs, attention_mask=causal_mask
        )
        out_1 = self.layernorm_1(inputs + attention_output_1)

        attention_output_2 = self.attention_2(
            query=out_1,
            value=encoder_outputs,
            key=encoder_outputs,
            attention_mask=padding_mask,
        )
        out_2 = self.layernorm_2(out_1 + attention_output_2)

        proj_output = self.dense_proj(out_2)
        return self.layernorm_3(out_2 + proj_output)

    def get_causal_attention_mask(self, inputs):
        input_shape = tf.shape(inputs)
        batch_size, sequence_length = input_shape[0], input_shape[1]
        i = tf.range(sequence_length)[:, None]
        j = tf.range(sequence_length)
        mask = tf.cast(i >= j, dtype="int32")
        mask = tf.reshape(mask, (1, input_shape[1], input_shape[1]))
        mult = tf.concat(
            [tf.expand_dims(batch_size, -1), tf.convert_to_tensor([1, 1])],
            axis=0,
        )
        return tf.tile(mask, mult)

    def get_config(self):
        config = super().get_config()
        config.update(
            {
                "embed_dim": self.embed_dim,
                "latent_dim": self.latent_dim,
                "num_heads": self.num_heads,
            }
        )
        return config


In [66]:
embed_dim = 256
latent_dim = 2048
num_heads = 8

encoder_inputs = keras.Input(shape=(None,), dtype="int64", name="encoder_inputs")
x = PositionalEmbedding(sequence_length, vocab_size, embed_dim)(encoder_inputs)
encoder_outputs = TransformerEncoder(embed_dim, latent_dim, num_heads)(x)
encoder = keras.Model(encoder_inputs, encoder_outputs)

decoder_inputs = keras.Input(shape=(None,), dtype="int64", name="decoder_inputs")
encoded_seq_inputs = keras.Input(shape=(None, embed_dim), name="decoder_state_inputs")
x = PositionalEmbedding(sequence_length, vocab_size, embed_dim)(decoder_inputs)
x = TransformerDecoder(embed_dim, latent_dim, num_heads)(x, encoded_seq_inputs)
x = layers.Dropout(0.5)(x)
decoder_outputs = layers.Dense(vocab_size, activation="softmax")(x)
decoder = keras.Model([decoder_inputs, encoded_seq_inputs], decoder_outputs)

decoder_outputs = decoder([decoder_inputs, encoder_outputs])
transformer = keras.Model(
    [encoder_inputs, decoder_inputs], decoder_outputs, name="transformer"
)

In [67]:
epochs = 10  # This should be at least 30 for convergence

transformer.summary()
transformer.compile(
    "rmsprop", loss="sparse_categorical_crossentropy", metrics=["accuracy"]
)
transformer.fit(train_ds, epochs=epochs, validation_data=val_ds)

Model: "transformer"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 encoder_inputs (InputLayer  [(None, None)]               0         []                            
 )                                                                                                
                                                                                                  
 positional_embedding_2 (Po  (None, None, 256)            3845120   ['encoder_inputs[0][0]']      
 sitionalEmbedding)                                                                               
                                                                                                  
 decoder_inputs (InputLayer  [(None, None)]               0         []                            
 )                                                                                      

In [68]:
import tensorflow as tf
import random

# Define the decoding function
def decode_sequence(input_sentence):
    tokenized_input_sentence = eng_vectorization([input_sentence])
    decoded_sentence = "[start]"
    for i in range(max_decoded_sentence_length):
        tokenized_target_sentence = spa_vectorization([decoded_sentence])[:, :-1]
        predictions = transformer([tokenized_input_sentence, tokenized_target_sentence])

        sampled_token_index = tf.argmax(predictions[0, i, :]).numpy().item(0)
        sampled_token = spa_index_lookup[sampled_token_index]
        decoded_sentence += " " + sampled_token

        if sampled_token == "[end]":
            break
    return decoded_sentence

# Define the maximum length for decoded sentences
max_decoded_sentence_length = 20

# Define spa_index_lookup
spa_vocab = spa_vectorization.get_vocabulary()
spa_index_lookup = dict(zip(range(len(spa_vocab)), spa_vocab))

# Define the English sentence to translate
english_sentence = "Deep Learning is widely used in Natural Language Processing, as Dr. Sun said in CSC 495/693."

# Call the decode_sequence function to translate the English sentence
translated_sentence = decode_sequence(english_sentence)
print("Translated Sentence:", translated_sentence)


Translated Sentence: [start] la [UNK] es que [UNK] en la lengua natural más viejo sol como el sol y la [UNK] end 


In [53]:
import tensorflow as tf
import random

# Define the decoding function
def decode_sequence(input_sentence):
    tokenized_input_sentence = eng_vectorization([input_sentence])
    decoded_sentence = "[start]"
    for i in range(max_decoded_sentence_length):
        tokenized_target_sentence = spa_vectorization([decoded_sentence])[:, :-1]
        predictions = transformer([tokenized_input_sentence, tokenized_target_sentence])

        sampled_token_index = tf.argmax(predictions[0, i, :]).numpy().item(0)
        sampled_token = spa_index_lookup[sampled_token_index]
        decoded_sentence += " " + sampled_token

        if sampled_token == "[end]":
            break
    return decoded_sentence

# Define the maximum length for decoded sentences
max_decoded_sentence_length = 20

# Define spa_index_lookup
spa_vocab = spa_vectorization.get_vocabulary()
spa_index_lookup = dict(zip(range(len(spa_vocab)), spa_vocab))
test_eng_texts = [pair[0] for pair in test_pairs]
for _ in range(30):
    input_sentence = random.choice(test_eng_texts)
    translated = decode_sequence(input_sentence)
    print("Input:", input_sentence)
    print("Translated:", translated)
    print()

Input: Tom wore a white jacket.
Translated: [start] tomás llevaba una campera blanca [end]

Input: Tom split up with Mary.
Translated: [start] tom terminó con mary [end]

Input: She could not understand the whole story.
Translated: [start] ella no entendía toda la historia [end]

Input: It is about the size of an egg.
Translated: [start] tiene alrededor del alto de un huevo [end]

Input: Tom is looking at us.
Translated: [start] tom nos está mirando [end]

Input: This isn't the first time it's snowed in Boston on Christmas Day.
Translated: [start] no es la primera vez que nieva con la navidad en boston en boston en boston [end]

Input: He was impatient to see his son.
Translated: [start] estaba impaciente por ver a su hijo [end]

Input: What are they hiding?
Translated: [start] qué se están ocultando [end]

Input: Can you show me how to tie a tie?
Translated: [start] puedes mostrarme cómo encontrar una corbata [end]

Input: He left the room.
Translated: [start] Él salió de la habitació

In [60]:
# Define a variable to keep track of the number of correct translations
num_correct_translations = 0

# Iterate over a random selection of 30 English sentences and their corresponding ground truth translations
for _ in range(30):
    input_sentence, ground_truth_translation = random.choice(test_pairs)
    translated = decode_sequence(input_sentence)

    # Print the input sentence, the ground truth translation, and the translated sentence
    print("Input:", input_sentence)
    print("Ground Truth:", ground_truth_translation)
    print("Translated:", translated)
    print()

    # Check if the translated sentence matches the ground truth translation
    if translated.strip() == ground_truth_translation.strip():
        num_correct_translations += 1

# Calculate the accuracy
accuracy = num_correct_translations / 30
print("Number of correct translations:", num_correct_translations)
print("Total sentences:", 30)
print("Accuracy:", accuracy)


Input: You have no legal right to seize my property.
Ground Truth: [start] Usted no tiene derecho legal a confiscar mi propiedad. [end]
Translated: [start] no tienes legal por [UNK] mis [UNK] [end]

Input: She wants a new hat.
Ground Truth: [start] Ella quiere un sombrero nuevo. [end]
Translated: [start] ella quiere un nuevo sombrero [end]

Input: She is more of an acquaintance than a friend.
Ground Truth: [start] Ella más que una amiga es una conocida. [end]
Translated: [start] ella es más de un conocido en un amigo [end]

Input: You start.
Ground Truth: [start] Tú empieza. [end]
Translated: [start] yo [UNK] [end]

Input: It may rain.
Ground Truth: [start] Puede que llueva. [end]
Translated: [start] puede que llueva [end]

Input: How was your afternoon?
Ground Truth: [start] ¿Cómo estuvo tu tarde? [end]
Translated: [start] qué tal tu tarde [end]

Input: Keep your eyes closed until I tell you to open them.
Ground Truth: [start] Cierra los ojos hasta que te diga que los abras. [end]
Tra